In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0


In [ ]:
import matplotlib.pyplot as plt

# Plot some sample images
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f'Label: {y_train[i]}')
    plt.axis('off')
plt.show()


In [ ]:
from tensorflow.keras import layers, models

def create_lenet_model():
    model = models.Sequential()
    model.add(layers.Reshape((28, 28, 1), input_shape=(28, 28)))
    model.add(layers.Conv2D(6, (5, 5), activation='tanh', padding='same'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(16, (5, 5), activation='tanh'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(120, (5, 5), activation='tanh'))
    model.add(layers.Flatten())
    model.add(layers.Dense(84, activation='tanh'))
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_lenet_model()


In [ ]:
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)


In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


In [ ]:
from tensorflow.keras.optimizers import Adam

def create_lenet_model_with_params(learning_rate=0.001, batch_size=64, num_neurons=84):
    model = models.Sequential()
    model.add(layers.Reshape((28, 28, 1), input_shape=(28, 28)))
    model.add(layers.Conv2D(6, (5, 5), activation='tanh', padding='same'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(16, (5, 5), activation='tanh'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(120, (5, 5), activation='tanh'))
    model.add(layers.Flatten())
    model.add(layers.Dense(num_neurons, activation='tanh'))
    model.add(layers.Dense(10, activation='softmax'))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Parameters to experiment with
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]
num_neurons_list = [64, 84, 128]

results = []

for lr in learning_rates:
    for batch_size in batch_sizes:
        for num_neurons in num_neurons_list:
            print(f"Training with learning_rate={lr}, batch_size={batch_size}, num_neurons={num_neurons}")
            model = create_lenet_model_with_params(learning_rate=lr, batch_size=batch_size, num_neurons=num_neurons)
            history = model.fit(x_train, y_train, epochs=5, batch_size=batch_size, validation_split=0.1, verbose=2)
            test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
            results.append({
                'learning_rate': lr,
                'batch_size': batch_size,
                'num_neurons': num_neurons,
                'test_loss': test_loss,
                'test_accuracy': test_accuracy
            })


In [ ]:
from tensorflow.keras import regularizers

def create_lenet_model_with_dropout_and_l2(dropout_rate=0.5, l2_reg=0.01):
    model = models.Sequential()
    model.add(layers.Reshape((28, 28, 1), input_shape=(28, 28)))
    model.add(layers.Conv2D(6, (5, 5), activation='tanh', padding='same', kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(16, (5, 5), activation='tanh', kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(120, (5, 5), activation='tanh', kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.Flatten())
    model.add(layers.Dense(84, activation='tanh', kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train with and without Dropout/L2
regularization_results = {}

# Without regularization
print("Training without regularization")
model_no_reg = create_lenet_model_with_params()
history_no_reg = model_no_reg.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1, verbose=2)
test_loss_no_reg, test_accuracy_no_reg = model_no_reg.evaluate(x_test, y_test, verbose=0)

# With Dropout and L2
print("Training with Dropout and L2 regularization")
model_reg = create_lenet_model_with_dropout_and_l2()
history_reg = model_reg.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1, verbose=2)
test_loss_reg, test_accuracy_reg = model_reg.evaluate(x_test, y_test, verbose=0)

regularization_results['no_regularization'] = (test_loss_no_reg, test_accuracy_no_reg)
regularization_results['with_regularization'] = (test_loss_reg, test_accuracy_reg)


In [ ]:
def plot_history(history, title):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'])

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'])
    plt.show()

plot_history(history_no_reg, 'Without Regularization')
plot_history(history_reg, 'With Regularization')


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Make predictions
y_pred = model_reg.predict(x_test)
y_pred_classes = tf.argmax(y_pred, axis=1)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(range(10)))
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()
